# Music Generation using LSTM 

Importing the necessary prerequisite libraries



In [1]:
import glob
import pickle
import numpy

In [2]:
from music21 import converter, instrument, note, chord

Creating the model using stacked Lstm. This means there are multiple layers of lstm inside the model handling learning therefore creating a robust model.The dropout layers provide regularisation and reduce overfitting. Softmax activation and categorical cross entropy is used to calculate the loss which measures the difference between the real next note and our predicted next note.

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

We load each file into a Music21 object using the converter.parse. This will parse the files and We will get a list of notes and chords.Using the Music21 .PartitionByInstrument function, the file is broken up into individual instruments, if there are multiple instruments. Each instrument contains a list of notes which are extracted and appended to a list. We can then save these into a file using pickle. We append the pitch of every note object using its string notation and every chord by encoding the id of every note in the chord. Now that we have put all the notes and chords into a sequential list, we create sequences that will work as an input.

In [4]:
final_notes = []
for file in glob.glob("../Dataset/midi_songs/*.mid"):
    midi = converter.parse(file)
    print("Ongoing converter parsing on file %s" % file)
    parsed_notes = None
    try: 
        s2 = instrument.partitionByInstrument(midi)
        parsed_notes = s2.parts[0].recurse() 
    except: 
        parsed_notes = midi.flat.notes
    for element in parsed_notes:
        if isinstance(element, note.Note):
            final_notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            final_notes.append('.'.join(str(n) for n in element.normalOrder))

with open("../Dataset/data/notes", 'wb') as filepath:
    pickle.dump(final_notes, filepath)

Ongoing converter parsing on file ../Dataset/midi_songs\0fithos.mid
Ongoing converter parsing on file ../Dataset/midi_songs\8.mid
Ongoing converter parsing on file ../Dataset/midi_songs\ahead_on_our_way_piano.mid
Ongoing converter parsing on file ../Dataset/midi_songs\AT.mid
Ongoing converter parsing on file ../Dataset/midi_songs\balamb.mid
Ongoing converter parsing on file ../Dataset/midi_songs\bcm.mid
Ongoing converter parsing on file ../Dataset/midi_songs\BlueStone_LastDungeon.mid
Ongoing converter parsing on file ../Dataset/midi_songs\braska.mid
Ongoing converter parsing on file ../Dataset/midi_songs\caitsith.mid
Ongoing converter parsing on file ../Dataset/midi_songs\Cids.mid
Ongoing converter parsing on file ../Dataset/midi_songs\cosmo.mid
Ongoing converter parsing on file ../Dataset/midi_songs\costadsol.mid
Ongoing converter parsing on file ../Dataset/midi_songs\dayafter.mid
Ongoing converter parsing on file ../Dataset/midi_songs\decisive.mid
Ongoing converter parsing on file ..

In [5]:
n_vocab = len(set(final_notes))

sequence_length = 100
sorted_set = sorted(set(item for item in final_notes))
note_to_int = dict((note, number) for number, note in enumerate(sorted_set))

network_input = []
network_output = []

    
for i in range(0, len(final_notes) - sequence_length, 1):
    sequence_in = final_notes[i:i + sequence_length] #creating input sequence with first 100 notes 
    sequence_out = final_notes[i + sequence_length] #creating output sequence with the remaining notes
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)


In [6]:
network_input

array([[[0.44480519],
        [0.90584416],
        [0.44480519],
        ...,
        [0.98376623],
        [0.7987013 ],
        [0.88961039]],

       [[0.90584416],
        [0.44480519],
        [0.44480519],
        ...,
        [0.7987013 ],
        [0.88961039],
        [0.11363636]],

       [[0.44480519],
        [0.44480519],
        [0.44480519],
        ...,
        [0.88961039],
        [0.11363636],
        [0.96428571]],

       ...,

       [[0.30519481],
        [0.30194805],
        [0.41883117],
        ...,
        [0.15909091],
        [0.98701299],
        [0.04545455]],

       [[0.30194805],
        [0.41883117],
        [0.73051948],
        ...,
        [0.98701299],
        [0.04545455],
        [0.86688312]],

       [[0.41883117],
        [0.73051948],
        [0.90909091],
        ...,
        [0.04545455],
        [0.86688312],
        [0.30194805]]])

In [7]:
model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),recurrent_dropout=0.3,return_sequences=True))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')



In [ ]:
model.fit(network_input, network_output, epochs=1, batch_size=128)

Generate notes from the neural network based on a sequence of notes

In [ ]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(sorted_set))
pattern = network_input[start]
prediction_output = []
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

convert the output from the prediction to notes and create a midi file from the notes

In [ ]:
offset = 0
output_notes = []
for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)

    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='output.mid')